# Tutorial 2: AUV navigation

In this tutorial you are going to learn:
 - how to work with auv_nav and what options are available
 - how to use the help menu available in the Command Line Interface (CLI)
 - how to process your own dataset with auv_nav to obtain a fused navigation solution
  
 To call our AUV navigation preprocessing software, you just need to write __auv_nav__ in a bash shell. You can also run it here:

In [1]:
%%bash --no-raise-error
auv_nav

  @@@@    @@@@      ___________________________ 
 @@@@@@  @@@@@@    |                           |
  @@@@    @@@@     |    - OCEAN PERCEPTION -   |
                   |                           |
  @@@@      @      | University of Southampton |
 @@@@@@   @@@@     |___________________________|
  @@@@   @@@@@@                                 
                                                
 Copyright (C) 2019 University of Southampton   
 This program comes with ABSOLUTELY NO WARRANTY.
 This is free software, and you are welcome to  
 redistribute it.                               
                                                
[INFO]: Running auv_nav version 0.0.3.0


usage: auv_nav [-h] {parse,process,convert} ...

positional arguments:
  {parse,process,convert}
    parse               Parse raw data and converting it to an intermediate
                        dataformat for further processing. Type auv_nav parse
                        -h for help on this target.
    process             Process and/or convert data. Data needs to be saved in
                        the intermediate data format generated using
                        auv_nav.py parse. Type auv_nav process -h for help on
                        this target.
    convert             Converts data from oplab nav_standard.json into your
                        specified output format. Type auv_nav convert -h for
                        help on this target.

optional arguments:
  -h, --help            show this help message and exit


## auv_nav workflow
As you can see, there are three processing possibilities with auv_nav:
 - __parse__: Reads the raw AUV navigation and sensor data and collates it into a standarised middle format in a file called *nav_standard.json*. This JSON file contains all sensor data ordered in time. At the end of this process you will see an informative table with the number of measurements read and a timeseries plot to spot any wrong time offset or timezone.
 - __process__: Reads the intermediate JSON file and produces a Dead Reckoning (DR) solution, as well as a fused solution from separate processes: an Extended Kalman Filter (EKF) and a Particle Filter (PF). At the end of this process you will see a set of interactive graphs showing the 2D navigation and the different velocities, depth, orientation and altitude vs time. The output of the processing step is a set of folders containing CSV navigation files for the vehicle, as well as for the cameras, if any. These files will be vital in the following processing pipelines developed by *ocean_perception*. 
 - __convert__: These optional step converts the JSON file into other formats, such as ACFR processing pipeline. If you need any other intermediate format, you can raise an issue in github or submit a pull request for us. 
 
All these options have their own help menu, which can be called by following the argument with a `-h`. See the example below.

In [2]:
%%bash --no-raise-error
auv_nav parse -h

  @@@@    @@@@      ___________________________ 
 @@@@@@  @@@@@@    |                           |
  @@@@    @@@@     |    - OCEAN PERCEPTION -   |
                   |                           |
  @@@@      @      | University of Southampton |
 @@@@@@   @@@@     |___________________________|
  @@@@   @@@@@@                                 
                                                
 Copyright (C) 2019 University of Southampton   
 This program comes with ABSOLUTELY NO WARRANTY.
 This is free software, and you are welcome to  
 redistribute it.                               
                                                
[INFO]: Running auv_nav version 0.0.3.0
usage: auv_nav parse [-h] [-F] path

positional arguments:
  path         Folderpath where the (raw) input data is. Needs to be a
               subfolder of 'raw' and contain the mission.yaml configuration
               file.

optional arguments:
  -h, --help   show this help message and exit
  -F, --Force  Force file o

## Example workflow
You are going to use *auv_nav* on the dataset you just download on Tutorial 1. We want you to work on the example dive dataset called `YYYYMMDD_hhmmss_platform_sensor_data`, which should be under `raw/year/cruise/platform` in your folder structure.

<div class="alert alert-warning">
<b>Note:</b> Please make sure to change the <font color="blue">dive_path</font> to a dive folder path in your computer.
</div>

In [1]:
dive_path = '/data/dives/raw/year/cruise/platform/YYYYMMDD_hhmmss_platform_sensor_data'

# We're going to check that the folder you provided is valid
from auv_nav.tools.folder_structure import valid_dive
if valid_dive(dive_path):
    print("[SUCCESS]: The provided folder is valid!")
else:
    print("[ ERROR ]: The provided folder is NOT valid. Check the path, it has to contain a mission and a vehicle yaml.")

[SUCCESS]: The provided folder is valid!


### Parsing

Now, call the *parse* step of auv_nav and explore its outputs:

<div class="alert alert-warning">
<b>Note:</b> If you call this script on a dive that has already been parsed, it will not overwrite the files unless you provide it the flag <b>-F</b>
</div>

In [8]:
!{'auv_nav parse -F ' + dive_path}

  @@@@    @@@@      ___________________________ 
 @@@@@@  @@@@@@    |                           |
  @@@@    @@@@     |    - OCEAN PERCEPTION -   |
                   |                           |
  @@@@      @      | University of Southampton |
 @@@@@@   @@@@     |___________________________|
  @@@@   @@@@@@                                 
                                                
 Copyright (C) 2019 University of Southampton   
 This program comes with ABSOLUTELY NO WARRANTY.
 This is free software, and you are welcome to  
 redistribute it.                               
                                                
[INFO]: Running auv_nav version 0.0.3.0
[INFO]: Loading mission.yaml at /data/dives/raw/year/cruise/platform/YYYYMMDD_hhmmss_platform_sensor_data/mission.yaml
[INFO]: Loading vehicle.yaml at /data/dives/raw/year/cruise/platform/YYYYMMDD_hhmmss_platform_sensor_data/vehicle.yaml
[WARN]: Depth standard deviation offset not provided. Using default of 0.01
[WARN]: A

[INFO]: Complete parse data


When finished, the parse step will write a `nav_standard.json` file with all the sensor measurements ordered in time and two HTML plots. Run the next cell to have a look at the sensor timestamps and to an informative table about the number of sensor measurements parsed.

In [5]:
from auv_nav.tools.folder_structure import get_processed_folder
from auv_nav.parse_data import check_output_files_exist

processed_dive_path = get_processed_folder(dive_path)
json_file = processed_dive_path / 'nav' / 'nav_standard.json'
data_plot_file = processed_dive_path / 'nav' / 'json_data_info.html'
history_plot_file = processed_dive_path / 'nav' / 'timestamp_history.html'

%mkdir -p plots
!{'ln -s ' + str(data_plot_file) +' plots/json_data_info.html'}
!{'ln -s ' + str(history_plot_file) +' plots/timestamp_history.html'}

if json_file.exists():
    print('The json file is at '+ str(json_file))
else:
    print('[ERROR]: The dataset has not been parsed yet.')

The json file is at /data/dives/processed/year/cruise/platform/YYYYMMDD_hhmmss_platform_sensor_data/nav/nav_standard.json


In [9]:
from IPython.display import IFrame
IFrame("plots/json_data_info.html", 1280, 1024)

In [12]:
IFrame('plots/timestamp_history.html', 800, 600)

## Processing the navigation
As you can see in the previous graphs,

Now, to process the navigation it is the same workflow, but with process instead of parse. Run the following cell:

<div class="alert alert-warning">
<b>Note:</b> If you call this script on a dive that has already been processed, it will not overwrite the files unless you provide it the flag <b>-F</b>
</div>

In [ ]:
!{'auv_nav process -F ' + str(dive_path)}

  @@@@    @@@@      ___________________________ 
 @@@@@@  @@@@@@    |                           |
  @@@@    @@@@     |    - OCEAN PERCEPTION -   |
                   |                           |
  @@@@      @      | University of Southampton |
 @@@@@@   @@@@     |___________________________|
  @@@@   @@@@@@                                 
                                                
 Copyright (C) 2019 University of Southampton   
 This program comes with ABSOLUTELY NO WARRANTY.
 This is free software, and you are welcome to  
 redistribute it.                               
                                                
[INFO]: Running auv_nav version 0.0.3.0
[INFO]: Loading existing auv_nav.yaml at /data/dives/configuration/year/cruise/platform/YYYYMMDD_hhmmss_platform_sensor_data/auv_nav.yaml
[INFO]: Loading vehicle.yaml
[INFO]: Using camera fore mounted at fore
[INFO]: Using camera aft mounted at aft
[INFO]: Using camera laser mounted at laser
[INFO]: Loading mission.yaml
[